In [1]:
import os
import json
from glob import glob
from tqdm import tqdm
from mmdet.apis import init_detector, inference_detector
from mmdet.utils import get_root_logger
get_root_logger().setLevel('NOTSET')

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Step1. 환경/경로 설정

In [2]:
OUTPUT_DIR = "./results"
IMAGE_DIR = "../dataset/images/test_open"
test_images = glob(os.path.join(IMAGE_DIR,"*.png"))
assert len(test_images) > 0, "데이터셋이 없습니다"

assert os.path.isdir(OUTPUT_DIR)

In [3]:
configs = [
    # 01: co_dino_5scale_swin_large_3x_thermal_aug1_train    epoch10           (val-mAP: 59.30, test_open-mAP: 50.21)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_train/co_dino_5scale_swin_large_3x_thermal_aug1_train_epoch10.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_train/co_dino_5scale_swin_large_3x_thermal_aug1_train.py"),
    # 02: co_dino_5scale_swin_large_3x_thermal_aug1_train    epoch10 w\ TTA    (val-mAP: -,     test_open-mAP: 50.97) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_train/co_dino_5scale_swin_large_3x_thermal_aug1_train_epoch10.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_train/co_dino_5scale_swin_large_3x_thermal_aug1_train-tta.py"),

    # 03. co_dino_5scale_swin_large_3x_thermal_aug2_train    epoch28           (val-mAP: 59.80, test_open-mAP: 50.04) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_train/co_dino_5scale_swin_large_3x_thermal_aug2_train_epoch28.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_train/co_dino_5scale_swin_large_3x_thermal_aug2_train.py"),
    # 04. co_dino_5scale_swin_large_3x_thermal_aug2_train    epoch28 w\ TTA    (val-mAP: -,     test_open-mAP: 50.76)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_train/co_dino_5scale_swin_large_3x_thermal_aug2_train_epoch28.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_train/co_dino_5scale_swin_large_3x_thermal_aug2_train-tta.py"),
    # 05. co_dino_5scale_swin_large_3x_thermal_aug2_train    epoch30           (val-mAP: 60.20, test_open-mAP: 50.01)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_train/co_dino_5scale_swin_large_3x_thermal_aug2_train_epoch30.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_train/co_dino_5scale_swin_large_3x_thermal_aug2_train.py"),

    # 06. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch27           (val-mAP: 56.70, test_open-mAP: 50.24) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train_epoch27.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train.py"),
    # 07. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch27 w\ TTA    (val-mAP: -,     test_open-mAP: 51.28)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train_epoch27.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train-tta.py"),
    # 08. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch31           (val-mAP: 56.90, test_open-mAP: 50.42) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train_epoch31.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train.py"),
    # 09. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch31 w\ TTA    (val-mAP: -,     test_open-mAP: 51.45)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train_epoch31.pth", "../workdirs/co_dino_5scale_swin_large_3x_thermal_aug3_train/co_dino_5scale_swin_large_3x_thermal_aug3_train-tta.py"),

    # 10. co_dino_5scale_swin_large_3x_thermal_aug1_trainval epoch15          (val-mAP: -,      test_open-mAP: 51.00) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_trainval/co_dino_5scale_swin_large_3x_thermal_aug1_trainval_epoch15.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_trainval/co_dino_5scale_swin_large_3x_thermal_aug1_trainval.py"),
    # 11. co_dino_5scale_swin_large_3x_thermal_aug1_trainval epoch15 w\ TTA   (val-mAP: -,      test_open-mAP: -)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_trainval/co_dino_5scale_swin_large_3x_thermal_aug1_trainval_epoch15.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_trainval/co_dino_5scale_swin_large_3x_thermal_aug1_trainval-tta.py"),
    # 12. co_dino_5scale_swin_large_3x_thermal_aug1_trainval epoch20          (val-mAP: -,      test_open-mAP: 51.04)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_trainval/co_dino_5scale_swin_large_3x_thermal_aug1_trainval_epoch20.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug1_trainval/co_dino_5scale_swin_large_3x_thermal_aug1_trainval.py"),

    # 13. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch19          (val-mAP: -,      test_open-mAP: 51.48) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch19.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval.py"),
    # 14. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch19 w\ TTA   (val-mAP: -,      test_open-mAP: 51.99)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch19.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval-tta.py"),
    # 15. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch21          (val-mAP: -,      test_open-mAP: 52.16) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch21.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval.py"),
    # 16. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch21 w\ TTA   (val-mAP: -,      test_open-mAP: 52.71)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch21.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval-tta.py"),
    # 17. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch25          (val-mAP: -,      test_open-mAP: 52.21) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch25.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval.py"),
    # 18. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch25 w\ TTA   (val-mAP: -,      test_open-mAP: 52.84)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch25.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval-tta.py"),
    # 19. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch27          (val-mAP: -,      test_open-mAP: 52.47)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch27.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval.py"),
    # 20. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch27 w\ TTA   (val-mAP: -,      test_open-mAP: 52.86)
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch27.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval-tta.py"),
    # 21. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch28          (val-mAP: -,      test_open-mAP: 52.36) 
    ("../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch28.pth","../workdirs/co_dino_5scale_swin_large_3x_thermal_aug2_trainval/co_dino_5scale_swin_large_3x_thermal_aug2_trainval.py"),
    
]

for (weight, config) in configs:
    assert os.path.isfile(weight), "weight 경로 오류"
    assert os.path.isfile(config), "config 경로 오류"

# Step2. 추론 수행 및 결과 저장 
(시간 오래 걸릴 수 있습니다.)

In [4]:
def inference(model, image_list):
    outputs = []
    for image_path in tqdm(image_list):
        basename = os.path.basename(image_path)
        result = inference_detector(model, image_path)
        # model.show_result(image_path, result, out_file=f"./tta_images/out_{basename}")
    
        for cat_id, _result in enumerate(result):
            for ann in _result:
                x1, y1, x2, y2, score = ann
                x, y, w, h = x1, y1, x2 - x1, y2 - y1
    
                if not score > 0.05:
                    continue
                ret = {
                    "image_id": basename[:-4],
                    "category_id": cat_id,
                    "bbox": [float(x),float(y),float(w),float(h)],
                    "score": float(score),
                }
                outputs.append(ret)
    return outputs

In [5]:
for weight, config in tqdm(configs, desc="Inference"):
    output_filename = os.path.basename(weight).strip('.pth')
    output_filename = f"{output_filename}-TTA.txt" if 'tta' in config else f"{output_filename}.txt"
    model = init_detector(config, weight, device='cuda')
    results = inference(model, test_images)

    assert os.path.isdir(OUTPUT_DIR)
    with open(os.path.join(OUTPUT_DIR, output_filename), 'w') as w:
        json.dump(results, w)
    

Inference:   0%|                                                                                 | 0/21 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755903507/work/aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
/workspace/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '


ema_backbone_layers_0_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_0_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_2_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_3_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_4_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_5_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_6_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_7_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_8_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_9_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_10_attn_relative_position_bias_table
ema_backbone_layers_2_bl


  0%|                                                                                             | 0/4 [00:00<?, ?it/s]/workspace/mmdet/datasets/utils.py:66: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")
/workspace/mmdet/models/utils/positional_encoding.py:81: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a

ema_backbone_layers_0_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_0_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_2_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_3_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_4_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_5_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_6_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_7_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_8_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_9_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_10_attn_relative_position_bias_table
ema_backbone_layers_2_bl


Inference:  10%|██████▉                                                                  | 2/21 [00:29<04:44, 14.99s/it]

...


Inference:  90%|█████████████████████████████████████████████████████████████████▏      | 19/21 [03:29<00:20, 10.35s/it]

ema_backbone_layers_0_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_0_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_2_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_3_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_4_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_5_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_6_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_7_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_8_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_9_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_10_attn_relative_position_bias_table
ema_backbone_layers_2_bl


Inference:  95%|████████████████████████████████████████████████████████████████████▌   | 20/21 [03:44<00:11, 11.84s/it]

ema_backbone_layers_0_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_0_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_1_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_0_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_1_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_2_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_3_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_4_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_5_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_6_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_7_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_8_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_9_attn_relative_position_bias_table
ema_backbone_layers_2_blocks_10_attn_relative_position_bias_table
ema_backbone_layers_2_bl


Inference: 100%|████████████████████████████████████████████████████████████████████████| 21/21 [03:51<00:00, 11.04s/it]


In [6]:
# Inference Info 
# 01: co_dino_5scale_swin_large_3x_thermal_aug1_train    epoch10           (val-mAP: 59.30, test_open-mAP: 50.21)
# 02: co_dino_5scale_swin_large_3x_thermal_aug1_train    epoch10 w\ TTA    (val-mAP: -,     test_open-mAP: 50.97) 

# 03. co_dino_5scale_swin_large_3x_thermal_aug2_train    epoch28           (val-mAP: 59.80, test_open-mAP: 50.04) 
# 04. co_dino_5scale_swin_large_3x_thermal_aug2_train    epoch28 w\ TTA    (val-mAP: -,     test_open-mAP: 50.76)
# 05. co_dino_5scale_swin_large_3x_thermal_aug2_train    epoch30           (val-mAP: 60.20, test_open-mAP: 50.01)

# 06. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch27           (val-mAP: 56.70, test_open-mAP: 50.24) 
# 07. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch27 w\ TTA    (val-mAP: -,     test_open-mAP: 51.28)
# 08. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch31           (val-mAP: 56.90, test_open-mAP: 50.42) 
# 09. co_dino_5scale_swin_large_3x_thermal_aug3_train    epoch31 w\ TTA    (val-mAP: -,     test_open-mAP: 51.45)

# 10. co_dino_5scale_swin_large_3x_thermal_aug1_trainval epoch15           (val-mAP: -,     test_open-mAP: 51.00) 
# 11. co_dino_5scale_swin_large_3x_thermal_aug1_trainval epoch15 w\ TTA    (val-mAP: -,     test_open-mAP: -)
# 12. co_dino_5scale_swin_large_3x_thermal_aug1_trainval epoch20           (val-mAP: -,     test_open-mAP: 51.04)

# 13. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch19           (val-mAP: -,     test_open-mAP: 51.48) 
# 14. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch19 w\ TTA    (val-mAP: -,     test_open-mAP: 51.99)
# 15. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch21           (val-mAP: -,     test_open-mAP: 52.16) 
# 16. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch21 w\ TTA    (val-mAP: -,     test_open-mAP: 52.71)
# 17. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch25           (val-mAP: -,     test_open-mAP: 52.21) 
# 18. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch25 w\ TTA    (val-mAP: -,     test_open-mAP: 52.84)
# 19. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch27           (val-mAP: -,     test_open-mAP: 52.47)
# 20. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch27 w\ TTA    (val-mAP: -,     test_open-mAP: 52.86)
# 21. co_dino_5scale_swin_large_3x_thermal_aug2_trainval epoch28           (val-mAP: -,     test_open-mAP: 52.36) 

# Ensenble test_open-mAP: 54.33
# ["w\ TTA", "trainval"] 는 val-mAP를 따로 측정하지 않았음 

In [7]:
!tree ./results

./results
├── co_dino_5scale_swin_large_3x_thermal_aug1_train_epoch10-TTA.txt
├── co_dino_5scale_swin_large_3x_thermal_aug1_train_epoch10.txt
├── co_dino_5scale_swin_large_3x_thermal_aug1_trainval_epoch15-TTA.txt
├── co_dino_5scale_swin_large_3x_thermal_aug1_trainval_epoch15.txt
├── co_dino_5scale_swin_large_3x_thermal_aug1_trainval_epoch20.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_train_epoch28-TTA.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_train_epoch28.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_train_epoch30.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch19-TTA.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch19.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch21-TTA.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch21.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch25-TTA.txt
├── co_dino_5scale_swin_large_3x_thermal_aug2_trainval_epoch25.txt
├── co_dino_5scale_swin_large_3x_thermal_au